<a href="https://colab.research.google.com/github/UEPP40/PUM/blob/matusiak/regresja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, median_absolute_error, r2_score

In [4]:
df  =  pd.read_csv("/content/Salary Data (2).csv")
df.dropna(inplace=True)
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


In [5]:
le = LabelEncoder()
columns= ["Gender", "Education Level", "Job Title"]
df[columns] = df[columns].apply(LabelEncoder().fit_transform)

df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,1,0,159,5.0,90000.0
1,28.0,0,1,17,3.0,65000.0
2,45.0,1,2,130,15.0,150000.0
3,36.0,0,0,101,7.0,60000.0
4,52.0,1,1,22,20.0,200000.0


In [6]:
train_df, test_df = train_test_split(df, test_size = 0.3, random_state=40)

X_train = train_df[['Age', 'Gender','Education Level', 'Job Title', 'Years of Experience']]
Y_train = train_df['Salary']
X_test = test_df[['Age', 'Gender', 'Education Level', 'Job Title', 'Years of Experience']]
Y_test = test_df['Salary']

In [8]:
model_linear = LinearRegression().fit(X_train,Y_train)
model_linear.score(X_train,Y_train)
model_linear.coef_
model_linear.intercept_
y_predic = model_linear.predict(X_test)

In [9]:
model_huber = HuberRegressor().fit(X_train,Y_train)
model_huber.score(X_train,Y_train)
model_huber.coef_
model_huber.intercept_
y_predic_huber = model_huber.predict(X_test)

In [10]:
y_true = Y_test
print ("Regresja liniowa: ", mean_absolute_error(y_true,y_predic))
print("Regresja Hubera: ", mean_absolute_error(y_true,y_predic_huber))

Regresja liniowa:  12351.261118548075
Regresja Hubera:  11696.811241388032


In [11]:
y_true = Y_test
print("Regresja liniowa: ", median_absolute_error(y_true,y_predic))
print("Regresja Hubera: ", median_absolute_error(y_true,y_predic_huber))

Regresja liniowa:  10532.659544538386
Regresja Hubera:  9092.997949978522


In [13]:
y_true = Y_test
print("Regresja liniowa: ", r2_score(y_true,y_predic))
print("Regresja Hubera: ", r2_score(y_true,y_predic_huber))

Regresja liniowa:  0.885420522400265
Regresja Hubera:  0.8944905861139743
